In [87]:
import os, gc
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import HistGradientBoostingRegressor

# ---------- CONFIG ----------
DATA_DIR = Path(".")
FILE_CONS  = DATA_DIR/"20201015_consumption.xlsx"
FILE_PROF  = DATA_DIR/"20201015_profiles.xlsx"
FILE_WEATH = DATA_DIR/"20201015_weather.xlsx"

# Keep a subset of meters initially to avoid RAM blow-ups; raise later
N_METERS_SAMPLE = 50                  # try 50 first; increase to e.g. 200 later
CHECKPOINT_LONG = DATA_DIR/"long_checkpoint.pkl"
USE_PARQUET_CHECKPOINT = True

# Optional: limit threads for native libs to reduce crashes on macOS
os.environ.setdefault("OMP_NUM_THREADS", "4")
os.environ.setdefault("KMP_DUPLICATE_LIB_OK", "TRUE")




'True'

In [88]:
import os, sys
print("CWD:", os.getcwd())
try:
    import lightgbm as lgb
    print("LightGBM:", lgb.__version__)
except Exception as e:
    print("LightGBM not importable:", e)

print("'model_lgb' exists?", 'model_lgb' in locals())
print("'gmodel' exists?", 'gmodel' in locals())
print("'hgb' exists?", 'hgb' in locals())


CWD: /Users/kaveengarthigeyan/Downloads/spanish-house-energy-consumption
LightGBM not importable: No module named 'lightgbm'
'model_lgb' exists? True
'gmodel' exists? False
'hgb' exists? True


In [89]:
def mem_gb(df): 
    return round(df.memory_usage(deep=True).sum()/1e9, 3)

def print_head(df, n=5, title=""):
    if title: print(f"\n--- {title} ---")
    print(df.head(n).to_string())

# ---------- 1) LOAD & RESHAPE ----------
if USE_PARQUET_CHECKPOINT and CHECKPOINT_LONG.exists():
    long = pd.read_pickle(CHECKPOINT_LONG)
    print(f"Loaded checkpoint: {CHECKPOINT_LONG} (GB={mem_gb(long)})")
else:
    print("Loading consumption (wide format)…")
    cons = pd.read_excel(FILE_CONS)
    cons.columns = cons.columns.map(str).str.strip()
    cons = cons.rename(columns={"date": "timestamp"})
    cons["timestamp"] = pd.to_datetime(cons["timestamp"], errors="coerce")
    print("Consumption shape:", cons.shape)

    # choose a manageable subset of IDs to start
    meter_cols = cons.columns.drop("timestamp")
    if N_METERS_SAMPLE and len(meter_cols) > N_METERS_SAMPLE:
        meter_cols = meter_cols[:N_METERS_SAMPLE]
    cons_small = cons[["timestamp"] + meter_cols.tolist()].copy()

    # wide -> long
    long = cons_small.melt(id_vars="timestamp",
                           var_name="household_id",
                           value_name="consumption_kwh")
    long = long.dropna(subset=["timestamp", "consumption_kwh"])
    long["household_id"] = long["household_id"].astype(str)
    long["consumption_kwh"] = long["consumption_kwh"].astype("float32")
    long.sort_values(["household_id","timestamp"], inplace=True, ignore_index=True)

    print("Long shape:", long.shape, "GB:", mem_gb(long))
    print_head(long, title="Long preview")

    # checkpoint to avoid redoing melt on every restart
    if USE_PARQUET_CHECKPOINT:
        long.to_pickle(CHECKPOINT_LONG)
        print(f"Checkpoint saved → {CHECKPOINT_LONG}")

gc.collect()


Loaded checkpoint: long_checkpoint.pkl (GB=0.037)


26345

In [90]:
print("\nFeature engineering…")
long.sort_values(["household_id","timestamp"], inplace=True, ignore_index=True)

# calendar
long["hour"] = long["timestamp"].dt.hour.astype("int16")
long["dow"] = long["timestamp"].dt.dayofweek.astype("int8")
long["is_weekend"] = (long["dow"] >= 5).astype("int8")

# cyclic encodings (helpful for daily/annual cycles)
long["hod_sin"] = np.sin(2*np.pi*long["hour"]/24).astype("float32")
long["hod_cos"] = np.cos(2*np.pi*long["hour"]/24).astype("float32")
doy = long["timestamp"].dt.dayofyear
long["doy_sin"] = np.sin(2*np.pi*doy/365).astype("float32")
long["doy_cos"] = np.cos(2*np.pi*doy/365).astype("float32")

# lags/rolling per household
grp = long.groupby("household_id")["consumption_kwh"]
long["lag_1h"]   = grp.shift(1)
long["lag_24h"]  = grp.shift(24)
long["lag_168h"] = grp.shift(168)
long["roll_mean_6h"] = grp.rolling(6, min_periods=1).mean().values
for c in ["lag_1h","lag_24h","lag_168h","roll_mean_6h"]:
    long[c] = long[c].astype("float32")

# usable training rows (need lag_168h)
model_df = long.dropna(subset=["lag_168h"]).copy()
print("Model_df shape:", model_df.shape, "GB:", mem_gb(model_df))
gc.collect()


print("\nBaseline (lag-168h)…")
model_df["baseline_pred"] = model_df["lag_168h"]
mae_baseline = mean_absolute_error(model_df["consumption_kwh"], model_df["baseline_pred"])
print("Baseline MAE (lag_168h):", mae_baseline)

# 3A) Daily seasonal naive (lag-24h)
model_df["baseline_day"] = model_df["lag_24h"]
mae_day = mean_absolute_error(model_df["consumption_kwh"], model_df["baseline_day"])
print("Baseline MAE (lag_24h / daily seasonal naive):", mae_day)

# 3B) ETS baseline (one representative household)
try:
    from statsmodels.tsa.holtwinters import ExponentialSmoothing
    hh = model_df["household_id"].value_counts().index[0]
    series = model_df.loc[model_df["household_id"] == hh, "consumption_kwh"]
    ets = ExponentialSmoothing(series, trend="add", seasonal="add", seasonal_periods=24)
    ets_fit = ets.fit()
    ets_forecast = ets_fit.forecast(len(series))
    mae_ets = mean_absolute_error(series, ets_forecast)
    print("ETS MAE (single household):", mae_ets)
except Exception as e:
    print("ETS baseline skipped:", repr(e))


Feature engineering…
Model_df shape: (429600, 14) GB: 0.055

Baseline (lag-168h)…
Baseline MAE (lag_168h): 9.846980094909668
Baseline MAE (lag_24h / daily seasonal naive): 4.190036773681641
ETS baseline skipped: ModuleNotFoundError("No module named 'statsmodels'")


In [91]:
# print("\nBaseline (lag-168h)…")
# model_df["baseline_pred"] = model_df["lag_168h"]
# mae_baseline = mean_absolute_error(model_df["consumption_kwh"], model_df["baseline_pred"])
# print("Baseline MAE:", mae_baseline)

# # ---------- 6) TRAIN FAST ON ONE HOUSEHOLD ----------
# print("\nTraining model on one household for speed…")
# sample_house = model_df["household_id"].value_counts().index[0]
# df_small = model_df.loc[model_df["household_id"] == sample_house].copy()
# df_small.sort_values("timestamp", inplace=True)

# features = [
#     "hour","dow","is_weekend","hod_sin","hod_cos","doy_sin","doy_cos",
#     "lag_1h","lag_24h","lag_168h","roll_mean_6h"
# ]
# X = df_small[features].astype("float32").fillna(0)
# y = df_small["consumption_kwh"].astype("float32")

# split = int(len(X)*0.8)
# X_train, X_test = X.iloc[:split], X.iloc[split:]
# y_train, y_test = y.iloc[:split], y.iloc[split:]

# # Stable, fast model (sklearn; no OpenMP headaches)
# hgb = HistGradientBoostingRegressor(
#     loss="absolute_error", learning_rate=0.05, max_iter=400,
#     early_stopping=True, validation_fraction=0.2, random_state=42
# )
# hgb.fit(X_train, y_train)
# preds_hgb = hgb.predict(X_test)
# mae_hgb = mean_absolute_error(y_test, preds_hgb)
# print("HGB MAE:", mae_hgb, "| Δ vs baseline:", mae_baseline - mae_hgb)

# # ---------- 7) OPTIONAL: LIGHTGBM (if installed) ----------
# try:
#     import lightgbm as lgb
#     print("\nTrying LightGBM…")
#     train_data = lgb.Dataset(X_train, label=y_train)
#     val_data   = lgb.Dataset(X_test,  label=y_test, reference=train_data)
#     params = {
#         "objective": "regression",
#         "metric": "mae",
#         "learning_rate": 0.05,
#         "num_leaves": 31,
#         "feature_fraction": 0.9,
#         "bagging_fraction": 0.9,
#         "bagging_freq": 5,
#         "verbosity": -1,
#         "num_threads": 4,
#     }
#     model = lgb.train(
#         params, train_data, num_boost_round=600, valid_sets=[val_data],
#         callbacks=[lgb.early_stopping(stopping_rounds=40), lgb.log_evaluation(100)]
#     )
#     preds_lgb = model.predict(X_test, num_iteration=model.best_iteration)
#     mae_lgb = mean_absolute_error(y_test, preds_lgb)
#     print("LightGBM MAE:", mae_lgb, "| Δ vs baseline:", mae_baseline - mae_lgb)
# except Exception as e:
#     print("LightGBM not run:", repr(e))

# # ---------- 8) SEGMENTED EVAL (great for slides) ----------
# print("\nSegmented evaluation (hour)…")
# test = df_small.iloc[split:].copy()
# test["pred_hgb"] = preds_hgb
# test["baseline"] = test["lag_168h"]
# test["err_model"] = (test["consumption_kwh"]-test["pred_hgb"]).abs()
# test["err_base"]  = (test["consumption_kwh"]-test["baseline"]).abs()
# seg = test.groupby("hour")[["err_model","err_base"]].mean().rename(
#     columns={"err_model":"MAE_model","err_base":"MAE_baseline"}
# )
# print(seg.head(24).to_string())

# print("\nDone.")


print("\nTraining model on one household for speed…")
sample_house = model_df["household_id"].value_counts().index[0]
df_small = model_df.loc[model_df["household_id"] == sample_house].copy()
df_small.sort_values("timestamp", inplace=True)

features = [
    "hour","dow","is_weekend","hod_sin","hod_cos","doy_sin","doy_cos",
    "lag_1h","lag_24h","lag_168h","roll_mean_6h"
]
X = df_small[features].astype("float32").fillna(0)
y = df_small["consumption_kwh"].astype("float32")

split = int(len(X)*0.8)
X_train, X_test = X.iloc[:split], X.iloc[split:]
y_train, y_test = y.iloc[:split], y.iloc[split:]

hgb = HistGradientBoostingRegressor(
    loss="absolute_error", learning_rate=0.05, max_iter=400,
    early_stopping=True, validation_fraction=0.2, random_state=42
)
hgb.fit(X_train, y_train)
preds_hgb = hgb.predict(X_test)
mae_hgb = mean_absolute_error(y_test, preds_hgb)
print("HGB MAE:", mae_hgb, "| Δ vs baseline:", mae_baseline - mae_hgb)

# ---------- 5) GLOBAL MODEL (multi-household) ----------
print("\nGlobal LightGBM model (across households)…")
mae_global = None
try:
    import lightgbm as lgb
    model_df["hh_te"] = model_df.groupby("household_id")["consumption_kwh"].transform("mean").astype("float32")
    features_global = ["hour","dow","is_weekend","lag_1h","lag_24h","lag_168h","roll_mean_6h","hh_te"]
    Xg = model_df[features_global].fillna(0).astype(np.float32)
    yg = model_df["consumption_kwh"].astype(np.float32)
    splitg = int(len(Xg)*0.8)
    Xg_train, Xg_test = Xg.iloc[:splitg], Xg.iloc[splitg:]
    yg_train, yg_test = yg.iloc[:splitg], yg.iloc[splitg:]
    gmodel = lgb.LGBMRegressor(objective="regression", learning_rate=0.05, n_estimators=300,
                               num_leaves=63, subsample=0.9, subsample_freq=5, colsample_bytree=0.9)
    gmodel.fit(Xg_train, yg_train, eval_set=[(Xg_test, yg_test)], eval_metric="l1",
               callbacks=[lgb.early_stopping(30), lgb.log_evaluation(50)])
    preds_g = gmodel.predict(Xg_test)
    mae_global = mean_absolute_error(yg_test, preds_g)
    print("Global LightGBM MAE:", mae_global)
except Exception as e:
    print("Global LightGBM skipped:", repr(e))

# ---------- 6) OPTIONAL: LightGBM (single-household point estimate) ----------
try:
    import lightgbm as lgb
    print("\nSingle-household LightGBM…")
    train_data = lgb.Dataset(X_train, label=y_train)
    val_data   = lgb.Dataset(X_test,  label=y_test, reference=train_data)
    params = {
        "objective": "regression",
        "metric": "mae",
        "learning_rate": 0.05,
        "num_leaves": 31,
        "feature_fraction": 0.9,
        "bagging_fraction": 0.9,
        "bagging_freq": 5,
        "verbosity": -1,
        "num_threads": 4,
    }
    model_lgb = lgb.train(
        params, train_data, num_boost_round=600, valid_sets=[val_data],
        callbacks=[lgb.early_stopping(stopping_rounds=40), lgb.log_evaluation(100)]
    )
    preds_lgb = model_lgb.predict(X_test, num_iteration=model_lgb.best_iteration)
    mae_lgb = mean_absolute_error(y_test, preds_lgb)
    print("LightGBM MAE:", mae_lgb, "| Δ vs baseline:", mae_baseline - mae_lgb)

    # ---------- 6A) Quantile LightGBM ----------
    print("\nQuantile LightGBM (P10/P50/P90)…")
    preds_q = {}
    for q in [0.1, 0.5, 0.9]:
        params_q = dict(objective="quantile", alpha=q, metric="quantile", learning_rate=0.05,
                        num_leaves=31, feature_fraction=0.9, bagging_fraction=0.9,
                        bagging_freq=5, verbosity=-1, num_threads=4)
        model_q = lgb.train(params_q, train_data, num_boost_round=400, valid_sets=[val_data],
                            callbacks=[lgb.early_stopping(40), lgb.log_evaluation(100)])
        preds_q[q] = model_q.predict(X_test, num_iteration=model_q.best_iteration)
    pi_width = (preds_q[0.9] - preds_q[0.1]).mean()
    coverage = np.mean((y_test >= preds_q[0.1]) & (y_test <= preds_q[0.9])) * 100
    print(f"Avg PI width: {pi_width:.5f} | Coverage: {coverage:.1f}%")

except Exception as e:
    print("LightGBM not run:", repr(e))
    model_lgb, preds_lgb = None, None


Training model on one household for speed…
HGB MAE: 0.010381790744524355 | Δ vs baseline: 9.836598304165143

Global LightGBM model (across households)…
Global LightGBM skipped: ModuleNotFoundError("No module named 'lightgbm'")
LightGBM not run: ModuleNotFoundError("No module named 'lightgbm'")


In [92]:
# import pandas as pd
# imp = pd.Series(model.feature_importance(), index=X.columns).sort_values(ascending=False)
# imp.head(10).plot(kind="barh", figsize=(6,4), title="Top-10 Feature Importances (LightGBM)")


print("\nSegmented evaluation (hour)…")
test = df_small.iloc[split:].copy()
test["pred_hgb"] = preds_hgb
test["baseline"] = test["lag_168h"]
test["err_model"] = (test["consumption_kwh"]-test["pred_hgb"]).abs()
test["err_base"]  = (test["consumption_kwh"]-test["baseline"]).abs()
seg = test.groupby("hour")[["err_model","err_base"]].mean().rename(
    columns={"err_model":"MAE_model","err_base":"MAE_baseline"}
)
print(seg.head(24).to_string())

print("\nExpanding-window backtest (HGB)…")
folds = [0.6, 0.7, 0.8]
fold_maes = []
for f in folds:
    cut = int(len(X) * f)
    m = HistGradientBoostingRegressor(
        loss="absolute_error", learning_rate=0.05, max_iter=400,
        early_stopping=True, validation_fraction=0.2, random_state=42
    )
    m.fit(X.iloc[:cut], y.iloc[:cut])
    preds_bt = m.predict(X.iloc[cut:])
    fold_maes.append(mean_absolute_error(y.iloc[cut:], preds_bt))
print("Backtest fold MAEs (HGB):", [round(x, 6) for x in fold_maes])



Segmented evaluation (hour)…
      MAE_model  MAE_baseline
hour                         
0      0.001775      0.000268
1      0.003847      0.003958
2      0.007451      0.010375
3      0.001774      0.000268
4      0.001404      0.000268
5      0.001359      0.000268
6      0.001292      0.000268
7      0.000680      0.000254
8      0.000103      0.000268
9      0.002482      0.002634
10     0.018675      0.023056
11     0.026350      0.027861
12     0.036087      0.076167
13     0.022722      0.044708
14     0.011916      0.034167
15     0.013520      0.036528
16     0.017211      0.005806
17     0.019458      0.036403
18     0.023660      0.056708
19     0.012924      0.044972
20     0.015552      0.000653
21     0.001591      0.000417
22     0.002575      0.000375
23     0.003663      0.000403

Expanding-window backtest (HGB)…
Backtest fold MAEs (HGB): [0.07441, 0.040037, 0.010382]


In [93]:
from matplotlib import pyplot as plt


print("\nResiduals & business impact…")
# Choose best preds available for plots:
preds_best = preds_hgb
if model_lgb is not None:
    preds_best = preds_lgb

residuals = y_test - preds_best
plt.figure(figsize=(6,4))
plt.scatter(preds_best, residuals, s=10, alpha=0.5)
plt.axhline(0, linestyle="--")
plt.title("Residuals vs Predicted")
plt.xlabel("Predicted"); plt.ylabel("Residuals")
plt.tight_layout(); plt.savefig("residuals_vs_predicted.png"); plt.close()

N = 10000  # scale example
avg_err_kwh = mean_absolute_error(y_test, preds_best)
daily_err_kwh = float(avg_err_kwh) * 24 * N
euro_per_kwh = 0.20
daily_cost = daily_err_kwh * euro_per_kwh
print(f"At {N} households, daily exposure ≈ €{daily_cost:,.0f}")


Residuals & business impact…
At 10000 households, daily exposure ≈ €498


In [94]:
# from matplotlib import pyplot as plt


# plt.figure(figsize=(10,4))
# plt.plot(y_test.values[:200], label="Actual")
# plt.plot(preds_hgb[:200], label="Predicted")
# plt.legend(); plt.title("Predicted vs Actual (HGB, sample 200 points)")


if 'model_lgb' in locals() and model_lgb is not None:
    try:
        imp = pd.Series(model_lgb.feature_importance(), index=X.columns).sort_values(ascending=False)
        ax = imp.head(10).plot(kind="barh", figsize=(6,4), title="Top-10 Feature Importances (LightGBM)")
        plt.tight_layout(); plt.savefig("feature_importance.png"); plt.close()
    except Exception as e:
        print("Feature importance plot skipped:", repr(e))

ax = seg.plot(kind="bar", figsize=(8,4), title="MAE by Hour (Model vs Baseline)")
plt.tight_layout(); plt.savefig("mae_by_hour.png"); plt.close()

plt.figure(figsize=(10,4))
plt.plot(y_test.values[:200], label="Actual")
plt.plot(preds_hgb[:200], label="Predicted (HGB)")
plt.legend(); plt.title("Predicted vs Actual (HGB, sample 200 points)")
plt.tight_layout(); plt.savefig("predicted_vs_actual.png"); plt.close()

# Fair baseline (non-NaN mask) check
baseline = model_df["consumption_kwh"].shift(168)
mask = ~baseline.isna()
mae_base2 = mean_absolute_error(model_df.loc[mask,"consumption_kwh"], baseline.loc[mask])
print("Recomputed baseline MAE (non-NaN rows):", mae_base2)

# ---------- 11) SAVE MODEL ----------
import joblib
joblib.dump({"model": hgb, "features": features, "baseline_mae": mae_baseline, "model_mae": mae_hgb},
            "energy_model.joblib")
print("\nDone.")

Recomputed baseline MAE (non-NaN rows): 9.870994567871094

Done.


In [95]:
baseline = model_df["consumption_kwh"].shift(168)
mask = ~baseline.isna()
mae_base2 = mean_absolute_error(model_df.loc[mask,"consumption_kwh"], baseline.loc[mask])
print("Recomputed baseline MAE:", mae_base2)


Recomputed baseline MAE: 9.870994567871094


In [96]:
import joblib
joblib.dump({
    "model": hgb,
    "features": features,
    "baseline_mae": mae_baseline,
    "model_mae": mae_hgb
}, "energy_model.joblib")


['energy_model.joblib']

In [97]:
# ============================================================
# Spanish household load forecasting — compact, robust pipeline
# Files expected (course zip):
#   - 20201015_consumption.xlsx  (wide: timestamp + household columns)
#   - 20201015_weather.xlsx      (wide: timestamp + *same* household columns OR station IDs)
#   - 20201015_profiles.xlsx     (maps household_id to optional weather/station id, varies by zip)
# Output:
#   - acf_plot.png
#   - corr_temp_consumption.png
#   - mae_by_hour.png
#   - residuals_vs_predicted.png
#   - energy_model.joblib
# ============================================================
import os, gc, numpy as np, pandas as pd
from pathlib import Path
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import HistGradientBoostingRegressor
import matplotlib.pyplot as plt

# -------- CONFIG --------
DATA_DIR = Path(".")
FILE_CONS  = DATA_DIR/"20201015_consumption.xlsx"
FILE_WEATH = DATA_DIR/"20201015_weather.xlsx"
FILE_PROF  = DATA_DIR/"20201015_profiles.xlsx"

# Start with a subset of meters to stay memory friendly; lift later
N_METERS_SAMPLE = 80

# Optional: limit threads to avoid crashing on some OSes
os.environ.setdefault("OMP_NUM_THREADS", "4")
os.environ.setdefault("KMP_DUPLICATE_LIB_OK", "TRUE")

def mem_gb(df): return round(df.memory_usage(deep=True).sum()/1e9, 3)

# -------- 1) LOAD CONSUMPTION (wide → long) --------
print("Loading consumption…")
cons = pd.read_excel(FILE_CONS)
cons.columns = cons.columns.map(str).str.strip()
cons = cons.rename(columns={"date": "timestamp"})
cons["timestamp"] = pd.to_datetime(cons["timestamp"], errors="coerce")

meter_cols = cons.columns.drop("timestamp")
if N_METERS_SAMPLE and len(meter_cols) > N_METERS_SAMPLE:
    meter_cols = meter_cols[:N_METERS_SAMPLE]
cons_small = cons[["timestamp"] + meter_cols.tolist()].copy()

long = cons_small.melt(
    id_vars="timestamp",
    var_name="household_id",
    value_name="consumption_kwh"
)
long = long.dropna(subset=["timestamp", "consumption_kwh"])
long["household_id"] = long["household_id"].astype(str)
long["consumption_kwh"] = long["consumption_kwh"].astype("float32")
long.sort_values(["household_id","timestamp"], inplace=True, ignore_index=True)
print("long shape:", long.shape, "GB:", mem_gb(long))

# -------- 2) CALENDAR FEATURES --------
long["hour"] = long["timestamp"].dt.hour.astype("int16")
long["dow"] = long["timestamp"].dt.dayofweek.astype("int8")
long["is_weekend"] = (long["dow"] >= 5).astype("int8")

# helpful cyclic encodings
long["hod_sin"] = np.sin(2*np.pi*long["hour"]/24).astype("float32")
long["hod_cos"] = np.cos(2*np.pi*long["hour"]/24).astype("float32")
doy = long["timestamp"].dt.dayofyear
long["doy_sin"] = np.sin(2*np.pi*doy/365).astype("float32")
long["doy_cos"] = np.cos(2*np.pi*doy/365).astype("float32")

# -------- 3) WEATHER: wide → long, join on correct key --------
def merge_weather(long_df: pd.DataFrame) -> pd.DataFrame:
    w = pd.read_excel(FILE_WEATH)
    w.columns = w.columns.map(str).str.strip()
    w = w.rename(columns={"date": "timestamp"})
    w["timestamp"] = pd.to_datetime(w["timestamp"], errors="coerce")
    print("[weather] columns:", list(w.columns)[:6], "...", f"(total {len(w.columns)})")

    # Are the non-time columns actually household IDs?
    non_time_cols = [c for c in w.columns if c != "timestamp"]
    set_cons_ids = set(long_df["household_id"].unique())
    overlap = set(c for c in non_time_cols if c in set_cons_ids)
    if overlap:
        # Great — the weather wide columns match household_id directly
        join_key = "household_id"
        w_long = w.melt(id_vars="timestamp", var_name=join_key, value_name="temperature")
        w_long[join_key] = w_long[join_key].astype(str)
        print(f"[weather] joining on (timestamp, {join_key}) with {len(w_long):,} rows.")
        return long_df.merge(w_long, on=["timestamp", join_key], how="left")

    # Else try to map via profiles (find the profiles column that overlaps)
    prof = None
    try:
        prof = pd.read_excel(FILE_PROF)
        prof.columns = prof.columns.map(str).str.strip()
        # pick the profiles col with best overlap with weather column names
        best_key = max(
            prof.columns,
            key=lambda c: len(set(prof[c].astype(str)) & set(non_time_cols))
        )
        overlap_cnt = len(set(prof[best_key].astype(str)) & set(non_time_cols))
        print(f"[weather] using profiles key '{best_key}' (overlap {overlap_cnt})")
        prof2 = prof[[best_key]].copy()
        prof2["household_id"] = prof2.index.astype(str) if "household_id" not in prof.columns else prof["household_id"].astype(str)
        prof2[best_key] = prof2[best_key].astype(str)
        prof2 = prof2.rename(columns={best_key: "weather_id"})
        # attach weather_id to long_df
        tmp = long_df.merge(prof2[["household_id","weather_id"]], on="household_id", how="left")
        w_long = w.melt(id_vars="timestamp", var_name="weather_id", value_name="temperature")
        w_long["weather_id"] = w_long["weather_id"].astype(str)
        return tmp.merge(w_long, on=["timestamp","weather_id"], how="left").drop(columns=["weather_id"])
    except Exception as e:
        print("[weather] profiles based join failed or not needed:", e)

    # Fallback: pick first numeric non-time column (not ideal, but better than failing)
    cand = next((c for c in non_time_cols if pd.api.types.is_numeric_dtype(w[c])), None)
    if cand is None:
        print("[weather] No usable weather column found; skipping temperature.")
        return long_df
    print(f"[weather] fallback: using '{cand}' for all rows (broadcast on timestamp only).")
    w2 = w.rename(columns={cand: "temperature"})[["timestamp","temperature"]]
    return long_df.merge(w2, on="timestamp", how="left")

long = merge_weather(long)

# -------- 4) LAGS & ROLLING --------
grp = long.groupby("household_id")["consumption_kwh"]
long["lag_1h"]   = grp.shift(1)
long["lag_24h"]  = grp.shift(24)
long["lag_168h"] = grp.shift(168)
long["roll_mean_6h"] = grp.rolling(6, min_periods=1).mean().values
for c in ["lag_1h","lag_24h","lag_168h","roll_mean_6h"]:
    long[c] = long[c].astype("float32")

# training frame (need lag_168h)
model_df = long.dropna(subset=["lag_168h"]).copy()
print("model_df:", model_df.shape, "GB:", mem_gb(model_df))
gc.collect()

# -------- 5) BASELINE --------
model_df["baseline_pred"] = model_df["lag_168h"]
mae_baseline = mean_absolute_error(model_df["consumption_kwh"], model_df["baseline_pred"])
print("Baseline MAE (lag_168h):", mae_baseline)

# -------- 6) TRAIN FAST MODEL (single household for speed) --------
sample_house = model_df["household_id"].value_counts().index[0]
df_small = model_df.loc[model_df["household_id"] == sample_house].sort_values("timestamp").copy()

features = [
    "hour","dow","is_weekend","hod_sin","hod_cos","doy_sin","doy_cos",
    "lag_1h","lag_24h","lag_168h","roll_mean_6h"
] + (["temperature"] if "temperature" in df_small.columns else [])

X = df_small[features].astype("float32").fillna(0)
y = df_small["consumption_kwh"].astype("float32")

split = int(len(X)*0.8)
X_train, X_test = X.iloc[:split], X.iloc[split:]
y_train, y_test = y.iloc[:split], y.iloc[split:]

hgb = HistGradientBoostingRegressor(
    loss="absolute_error", learning_rate=0.05, max_iter=400,
    early_stopping=True, validation_fraction=0.2, random_state=42
)
hgb.fit(X_train, y_train)
preds_hgb = hgb.predict(X_test)
mae_hgb = mean_absolute_error(y_test, preds_hgb)
print(f"HGB MAE: {mae_hgb:.6f} | Δ vs baseline: {mae_baseline - mae_hgb:.6f}")

# -------- 7) SEGMENTED EVAL (hour) & SAVE BARPLOT --------
test = df_small.iloc[split:].copy()
test["pred_hgb"] = preds_hgb
test["baseline"] = test["lag_168h"]
test["err_model"] = (test["consumption_kwh"]-test["pred_hgb"]).abs()
test["err_base"]  = (test["consumption_kwh"]-test["baseline"]).abs()
seg = test.groupby("hour")[["err_model","err_base"]].mean().rename(
    columns={"err_model":"MAE_model","err_base":"MAE_baseline"}
)

plt.figure(figsize=(12,6))
seg.plot(kind="bar", ax=plt.gca())
plt.title("Average Mean Absolute Error by Hour")
plt.xlabel("Hour of Day"); plt.ylabel("Mean Absolute Error (kWh)")
plt.legend(["HGB Model", "Baseline (lag_168h)"])
plt.tight_layout(); plt.savefig("mae_by_hour.png", dpi=300); plt.close()
print("Saved mae_by_hour.png")

# -------- 8) AUTOCORRELATION (NumPy, no statsmodels) & SAVE --------
series = df_small["consumption_kwh"].astype(float).to_numpy()
series = series[np.isfinite(series)]
series = series - series.mean()
max_lag = 200
acf_full = np.correlate(series, series, mode='full')
acf = acf_full[acf_full.size//2:][:max_lag+1] / acf_full[acf_full.size//2]

plt.figure(figsize=(12,6))
plt.bar(np.arange(len(acf)), acf, width=0.9)
plt.title("Autocorrelation of Hourly Electricity Consumption")
plt.xlabel("Lag (hours)"); plt.ylabel("Autocorrelation")
plt.tight_layout(); plt.savefig("acf_plot.png", dpi=300); plt.close()
print("Saved acf_plot.png")

# -------- 9) TEMP vs CONSUMPTION (hexbin) & SAVE --------
if "temperature" in model_df.columns:
    dfp = model_df[["temperature","consumption_kwh"]].dropna()
    # drop obviously nonsense values for a readable plot (does not affect training)
    dfp = dfp[dfp["consumption_kwh"].between(0, 25)]
    plt.figure(figsize=(8,6))
    hb = plt.hexbin(dfp["temperature"], dfp["consumption_kwh"], gridsize=60, mincnt=5, cmap="Blues")
    plt.colorbar(hb, label="count")
    plt.title("Temperature vs Electricity Consumption (household-hour)")
    plt.xlabel("Temperature (°C)"); plt.ylabel("Consumption (kWh)")
    plt.tight_layout(); plt.savefig("corr_temp_consumption.png", dpi=300); plt.close()
    print("Saved corr_temp_consumption.png")
else:
    print("Skipped corr_temp_consumption.png (no temperature column after merge).")

# -------- 10) RESIDUALS vs PREDICTED (scatter) & SAVE --------
residuals = y_test - preds_hgb
plt.figure(figsize=(9,5))
plt.scatter(preds_hgb, residuals, s=12, alpha=0.45)
plt.axhline(0, color="black", linestyle="--")
plt.title("Residuals vs Predicted Consumption")
plt.xlabel("Predicted Consumption (kWh)"); plt.ylabel("Residuals (kWh)")
plt.tight_layout(); plt.savefig("residuals_vs_predicted.png", dpi=300); plt.close()
print("Saved residuals_vs_predicted.png")

# -------- 11) SAVE MODEL SNIPPET --------
import joblib
joblib.dump({
    "model": hgb,
    "features": features,
    "baseline_mae": float(mae_baseline),
    "model_mae": float(mae_hgb),
    "household_id": sample_house
}, "energy_model.joblib")
print("Saved energy_model.joblib")


Loading consumption…
long shape: (700800, 3) GB: 0.06
[weather] columns: ['timestamp', '5d6fcd1cf44b0324bc6b7254', '5d6fcd1cf44b0324bc6b7257', '5d6fcd1cf44b0324bc6b725a', '5d6fcd1cf44b0324bc6b725d', '5d6fcd1df44b0324bc6b7260'] ... (total 500)
[weather] joining on (timestamp, household_id) with 4,371,240 rows.
model_df: (687520, 15) GB: 0.094
Baseline MAE (lag_168h): 6.238852024078369
HGB MAE: 0.010740 | Δ vs baseline: 6.228112
Saved mae_by_hour.png
Saved acf_plot.png
Saved corr_temp_consumption.png
Saved residuals_vs_predicted.png
Saved energy_model.joblib


In [98]:

# ==== 0. Setup ====
import os, numpy as np, pandas as pd
import matplotlib.pyplot as plt

os.makedirs("Images", exist_ok=True)

# Helper to nice-save
def savefig(path):
    plt.tight_layout()
    plt.savefig(path, dpi=300)
    plt.close()

# ==== 1) ACF without statsmodels (fast, no extra deps) ====
def acf(series, max_lag=200):
    x = series.values.astype(float)
    x = x - x.mean()
    denom = (x**2).sum()
    ac = [1.0]
    for k in range(1, max_lag+1):
        num = (x[:-k] * x[k:]).sum()
        ac.append(num/denom)
    return np.arange(0, max_lag+1), np.array(ac)

# Use one long household (or aggregate) for a clean ACF
ts = (df.sort_values(["household_id","timestamp"])
        .groupby("timestamp")["consumption_kwh"].mean())  # city/overall average
lags, ac_vals = acf(ts, max_lag=200)

plt.figure(figsize=(12,4))
plt.bar(lags[1:], ac_vals[1:], width=0.9)
plt.title("Autocorrelation of Hourly Electricity Consumption")
plt.xlabel("Lag (hours)"); plt.ylabel("Autocorrelation")
savefig("Images/acf_plot.png")

# ==== 2) Temperature vs consumption (hexbin for dense data) ====
plt.figure(figsize=(8,6))
hb = plt.hexbin(df["temperature"], df["consumption_kwh"], gridsize=60, mincnt=1)
plt.colorbar(hb, label="count")
plt.title("Temperature vs Electricity Consumption (household-hour)")
plt.xlabel("Temperature (°C)"); plt.ylabel("Consumption (kWh)")
savefig("Images/corr_temp_consumption.png")

# ==== 3) Residuals vs predicted ====
pred_df["resid"] = pred_df["y_true"] - pred_df["y_pred"]
plt.figure(figsize=(10,6))
plt.scatter(pred_df["y_pred"], pred_df["resid"], s=14, alpha=0.35)
plt.axhline(0, ls="--", c="k")
plt.title("Residuals vs Predicted Consumption")
plt.xlabel("Predicted Consumption (kWh)"); plt.ylabel("Residuals (kWh)")
savefig("Images/residuals_vs_predicted.png")

# ==== 4) MAE by hour: model vs baseline ====
mae_hour = (pred_df.assign(
    err_hgb=(pred_df["y_true"]-pred_df["y_pred"]).abs(),
    err_base=(pred_df["y_true"]-pred_df["y_base"]).abs()
).groupby("hour")[["err_hgb","err_base"]].mean())

plt.figure(figsize=(12,5))
plt.bar(mae_hour.index-0.2, mae_hour["err_hgb"], width=0.4, label="HGB Model")
plt.bar(mae_hour.index+0.2, mae_hour["err_base"], width=0.4, label="Baseline (lag_168h)")
plt.xticks(range(24)); plt.legend()
plt.title("Average Mean Absolute Error by Hour")
plt.xlabel("Hour of Day"); plt.ylabel("Mean Absolute Error (kWh)")
savefig("Images/mae_by_hour.png")

# ==== 5) Predicted vs actual for one sample series (pretty line plot) ====
sample_ids = pred_df["household_id"].dropna().unique()
hh = sample_ids[0]
ex = pred_df[pred_df["household_id"]==hh].sort_values("timestamp").tail(24*4)   # last 4 days
plt.figure(figsize=(12,4))
plt.plot(ex["timestamp"], ex["y_true"], label="Actual", lw=2)
plt.plot(ex["timestamp"], ex["y_pred"], label="HGB", lw=2, alpha=0.9)
plt.plot(ex["timestamp"], ex["y_base"], label="Baseline", lw=1.5, alpha=0.7)
plt.title(f"Predicted vs Actual (household {hh})")
plt.xlabel("Time"); plt.ylabel("Consumption (kWh)"); plt.legend()
savefig("Images/predicted_vs_actual.png")

# ==== 6) Feature importance (HGB supports it) ====
if hasattr(hgb, "feature_importances_"):
    fi = pd.Series(hgb.feature_importances_, index=hgb.feature_names_in_).sort_values(ascending=True).tail(20)
    plt.figure(figsize=(8,6))
    plt.barh(fi.index, fi.values)
    plt.title("Feature Importance (HGB)")
    plt.xlabel("Importance")
    savefig("Images/feature_importance.png")

# ==== 7) Baseline vs Model error comparison across households ====
err_by_hh = (pred_df.assign(
    mae_hgb=(pred_df["y_true"]-pred_df["y_pred"]).abs(),
    mae_base=(pred_df["y_true"]-pred_df["y_base"]).abs()
).groupby("household_id")[["mae_hgb","mae_base"]].mean().sort_values("mae_base"))

plt.figure(figsize=(10,5))
plt.plot(err_by_hh.index, err_by_hh["mae_base"], label="Baseline", lw=1.5)
plt.plot(err_by_hh.index, err_by_hh["mae_hgb"], label="HGB", lw=1.5)
plt.xticks([], [])  # many households: hide tick clutter
plt.title("Error comparison across households (MAE)")
plt.ylabel("MAE (kWh)"); plt.legend()
savefig("Images/error_comparison.png")

# ==== 8) Business Value visuals ====
# Configure these to your case:
C = 0.10   # $ per kWh absolute error (example)
MWh = 1.0  # average hourly load per household (MWh) or total portfolio MWh per hour
OH = pred_df["timestamp"].nunique()  # number of forecast hours in eval set

# Costs based on MAE (model vs baseline)
mae_model = (pred_df["y_true"]-pred_df["y_pred"]).abs().mean()
mae_base  = (pred_df["y_true"]-pred_df["y_base"]).abs().mean()

cost_model = mae_model * C * MWh * OH
cost_base  = mae_base  * C * MWh * OH
savings    = cost_base - cost_model

# 8a) Cost vs. model chart
plt.figure(figsize=(7,5))
plt.bar(["Baseline cost","Model cost"], [cost_base, cost_model], color=["#d37","#37d"])
plt.title("Error-Cost Comparison"); plt.ylabel("Total cost ($)")
for i,v in enumerate([cost_base, cost_model]):
    plt.text(i, v, f"${v:,.0f}", ha="center", va="bottom")
savefig("Images/business_cost_comparison.png")

# 8b) Cumulative savings over time
pred_df = pred_df.sort_values("timestamp")
inst_base  = (pred_df["y_true"]-pred_df["y_base"]).abs()  * C * MWh
inst_model = (pred_df["y_true"]-pred_df["y_pred"]).abs() * C * MWh
cum_sav = (inst_base - inst_model).groupby(pred_df["timestamp"]).mean().cumsum()

plt.figure(figsize=(10,4))
plt.plot(cum_sav.index, cum_sav.values)
plt.title("Cumulative Savings from Model vs Baseline")
plt.xlabel("Time"); plt.ylabel("Cumulative $")
savefig("Images/business_cumulative_savings.png")

# 8c) Sensitivity to price and volume (tornado-style simple)
scen = []
for c in [C*0.5, C, C*1.5]:
    scen.append((c, MWh, (mae_base-mae_model)*c*MWh*OH))
for m in [MWh*0.7, MWh, MWh*1.3]:
    scen.append((C, m, (mae_base-mae_model)*C*m*OH))
sens = pd.DataFrame(scen, columns=["C","MWh","Savings"])
labels = ["C -50%","C","C +50%","MWh -30%","MWh","MWh +30%"]

plt.figure(figsize=(7,5))
plt.barh(labels, sens["Savings"])
plt.title("Sensitivity of Savings to Price (C) and Volume (MWh)")
plt.xlabel("Savings ($)")
savefig("Images/business_sensitivity.png")


NameError: name 'df' is not defined